Authors:
    <br>Alejandro Alvarez (axa)
    <br>Brenda Palma (bpalmagu)

# <center>ML-Jokes: Content-Based Filtering</center>

In [1]:
# Path to ml-jokes folder
import os
if os.getcwd().split('/')[-2] == 'ml-jokes': os.chdir('..')
print(f'Current directory: {os.getcwd()}')
assert set(['data', 'mljokes', 'environment.yml', 'nbs']) <= set(os.listdir()), \
    'Wrong path; go to ./heinz-95729-project/api/ml-jokes'

Current directory: /home/alejandroxag/my_files/heinz-95729-project/api/ml-jokes


In [2]:
# imports
import optuna
import pickle
import numpy as np
import pandas as pd
from mljokes.topics import get_lda_topics
from sklearn.metrics import mean_absolute_error
from mljokes.data import read_jokes, read_ratings
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score

# Regression model inputs

In [72]:
def get_model_inputs(val_size, test_size, sample_size, n_topics, random_state):
    jokes_df = read_jokes()
    ratings_df = read_ratings()

    # Topics
    topics = get_lda_topics(text=jokes_df.clean_text, 
                                        ids=jokes_df.joke_id,
                                        n_topics=n_topics, 
                                        random_state=random_state)
    topics.rename(columns={'id': 'joke_id'}, inplace=True)

    # User-joke information
    user_joke_info = ratings_df.copy()
    user_joke_info.insert(4, 'unseen_joke', (user_joke_info.rating == 99.0).astype(int))

    test_examples = user_joke_info.index[user_joke_info.unseen_joke == 0]
    test_examples = np.random.choice(test_examples,
                                    size=round(test_size * len(test_examples)),
                                    replace=False)
    user_joke_info.insert(5, 
                          'test_example', 
                          user_joke_info.index.isin(test_examples).astype(int))

    val_examples = user_joke_info.index[(user_joke_info.unseen_joke + \
                                        user_joke_info.test_example) == 0]
    val_examples = np.random.choice(val_examples,
                                    size=round(val_size * len(val_examples)),
                                    replace=False)
    user_joke_info.insert(6, 
                          'val_example', 
                          user_joke_info.index.isin(val_examples).astype(int))

    user_joke_info.insert(7, 
                          'training_example', 
                          ((user_joke_info.unseen_joke + \
                            user_joke_info.test_example + \
                            user_joke_info.val_example) == 0).astype(int))

    user_joke_info = user_joke_info.merge(topics, how='left', on='joke_id')
    user_joke_info.rename(columns=dict(zip(user_joke_info.columns[8:],
                                          ['topic_percentage_' + c  \
                                            for c in np.array(user_joke_info.columns[8:], 
                                                              dtype=str)])), 
                          inplace=True)
    user_joke_info.reset_index(drop=True, inplace=True)

    user_joke_info = user_joke_info.merge(jokes_df.loc[:, ['joke_id', 'len']], how='left', on='joke_id')

    user_joke_info = user_joke_info.loc[:, list(user_joke_info.columns[:3]) + \
                                       ['len'] + \
                                       list(user_joke_info.columns[3:-1])]

    if sample_size < 1.0:
        user_joke_info = user_joke_info.iloc[np.random.randint(0, 
                                                               len(user_joke_info), 
                                                               size=round(sample_size * \
                                                                          len(user_joke_info))), :]

    user_joke_info_training = user_joke_info[(user_joke_info.unseen_joke + \
                                          user_joke_info.test_example + \
                                          user_joke_info.val_example) == 0]
    
    user_joke_info_training.drop(columns=['test_user', 
                                          'unseen_joke', 
                                          'test_example', 
                                          'val_example', 
                                          'training_example'], 
                                inplace=True)

    user_profiles = user_joke_info_training.iloc[:, :2]
    user_profiles.loc[:, list(range(len(user_joke_info_training.columns[4:])))] = \
        (user_joke_info_training.rating / 10).values.reshape((-1, 1)) * \
        user_joke_info_training.iloc[:, 4:].values
    user_profiles = user_profiles.groupby(['user_id'])[user_profiles.columns[2:]].agg('mean').reset_index()
    user_profiles.rename(columns=dict(zip(user_profiles.columns[1:],
                                          ['topic_user_rating_' + c \
                                          for c in np.array(user_profiles.columns[1:], dtype=str)])), 
                        inplace=True)
    user_joke_info_training = user_joke_info_training.merge(user_profiles, how='left', on='user_id')

    user_joke_info = user_joke_info.merge(user_joke_info_training \
                                            .iloc[:, [0] + list(-np.arange(n_topics, 0, -1))] \
                                            .drop_duplicates(),
                                        how='left', on='user_id')

    user_joke_info = user_joke_info.loc[~np.logical_and(user_joke_info.topic_user_rating_0.isna(), 
                                                    user_joke_info.val_example + \
                                                    user_joke_info.test_example + \
                                                    user_joke_info.unseen_joke == 1), :]
    
    user_joke_info.sort_values(by=['user_id', 'joke_id'], inplace=True)
    user_joke_info.reset_index(drop=True, inplace=True)
    features = ['len'] + list(user_joke_info.columns[9:])
    
    train_idxs = user_joke_info.index[user_joke_info.training_example + user_joke_info.val_example == 1]
    test_idxs = user_joke_info.index[user_joke_info.test_example == 1]

    folds = user_joke_info.loc[train_idxs, ['training_example', 'val_example']].reset_index(drop=True)
    folds = [(folds.index[folds.training_example == 1], folds.index[folds.val_example == 1])]

    y_train = user_joke_info.rating[train_idxs].values
    X_train = user_joke_info.loc[train_idxs, features].values
    y_test = user_joke_info.rating[test_idxs].values
    X_test = user_joke_info.loc[test_idxs, features].values

    return {'user_joke_info': user_joke_info,
            'features': features,
            'X_train': X_train,
            'y_train': y_train,
            'X_test': X_test,
            'y_test': y_test,
            'folds': folds,
            'jokes_df': jokes_df,
            'topics': topics,}

# Regression model (HGB)

### Optuna hypertuning

In [69]:
def tune(objective, n_trials, file_name, save_path='./results'):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")

    with open(f'{save_path}/{file_name}.pkl', 'wb') as f:
        pickle.dump(params, f)

    return params

def hgb_objective(trial):
    _learning_rate = trial.suggest_loguniform('learning_rate', low=1e-3, high=0.25)
    _max_depth = trial.suggest_int('max_depth', low=5, high=20, log=True)
    _l2_regularization = trial.suggest_loguniform('l2_regularization', low=1e-6, high=1)
    _min_samples_leaf = trial.suggest_int('min_samples_leaf', low=100, high=1_000)
    _tol = trial.suggest_loguniform('tol', low=1e-2, high=1e-1)
    _n_iter_no_change = trial.suggest_int('n_iter_no_change', low=10, high=100)
    _random_state = trial.suggest_int('random_state', 0, 1_000_000_000)

    fixed_params = {
                    'max_iter': 100_000,
                    'loss': 'absolute_error',
                    'max_leaf_nodes': None,
                    'early_stopping': True,
                    'scoring': 'loss',
                    'validation_fraction': 0.2,
                    # 'tol': 5e-2,
                    # 'n_iter_no_change': 50,
                    'verbose': 4,
                  }

    hgb = HistGradientBoostingRegressor(**fixed_params)
    
    hgb.set_params(**{
                      'learning_rate': _learning_rate,
                      'max_depth': _max_depth,
                      'l2_regularization': _l2_regularization,
                      'min_samples_leaf': _min_samples_leaf,
                      'tol': _tol,
                      'n_iter_no_change': _n_iter_no_change,
                      'random_state': _random_state
                      })

    print(f'\n\nCurrent model:\n\n{hgb.get_params()}\n')

    X = model_inputs['X_train']
    y = model_inputs['y_train']
    train_val_folds = model_inputs['folds']
    scores = cross_val_score(hgb, X, y, 
                             cv=train_val_folds, 
                             scoring="neg_mean_absolute_error")

    return scores.mean()

In [46]:
test_size = 0.15
val_size = 0.15
sample_size = 0.25
n_topics = 10
random_state = 0

model_inputs = get_model_inputs(val_size, test_size, sample_size, n_topics, random_state)

/home/alejandroxag/miniconda3/envs/ecommerce_jokes/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [56]:
hgb_params = tune(hgb_objective, n_trials=20, file_name='test_optuna2')

[I 2021-11-27 13:44:57,225] A new study created in memory with name: no-name-58c37c4f-8496-41d6-b2b6-44b6f8629dfc




Current model:

{'categorical_features': None, 'early_stopping': True, 'l2_regularization': 2.8204642290576796e-05, 'learning_rate': 0.044102916305382434, 'loss': 'absolute_error', 'max_bins': 255, 'max_depth': 6, 'max_iter': 100000, 'max_leaf_nodes': None, 'min_samples_leaf': 173, 'monotonic_cst': None, 'n_iter_no_change': 62, 'random_state': 830205047, 'scoring': 'loss', 'tol': 0.08059856466264227, 'validation_fraction': 0.2, 'verbose': 4, 'warm_start': False}

Binning 0.100 GB of training data: 0.858 s
Binning 0.025 GB of validation data: 0.126 s
Fitting gradient boosted rounds:
[1/100000] 1 tree, 64 leaves, max depth = 6, train loss: 4.32955, val loss: 4.33520, in 0.174s
[2/100000] 1 tree, 64 leaves, max depth = 6, train loss: 4.25298, val loss: 4.25839, in 0.160s
[3/100000] 1 tree, 64 leaves, max depth = 6, train loss: 4.18105, val loss: 4.18639, in 0.395s
[4/100000] 1 tree, 64 leaves, max depth = 6, train loss: 4.11311, val loss: 4.11852, in 0.221s
[5/100000] 1 tree, 64 leaves,

[I 2021-11-27 13:45:29,962] Trial 0 finished with value: -3.8876998387783153 and parameters: {'learning_rate': 0.044102916305382434, 'max_depth': 6, 'l2_regularization': 2.8204642290576796e-05, 'min_samples_leaf': 173, 'tol': 0.08059856466264227, 'n_iter_no_change': 62, 'random_state': 830205047}. Best is trial 0 with value: -3.8876998387783153.




Current model:

{'categorical_features': None, 'early_stopping': True, 'l2_regularization': 0.02168029753677356, 'learning_rate': 0.001415445349883366, 'loss': 'absolute_error', 'max_bins': 255, 'max_depth': 12, 'max_iter': 100000, 'max_leaf_nodes': None, 'min_samples_leaf': 160, 'monotonic_cst': None, 'n_iter_no_change': 79, 'random_state': 812124917, 'scoring': 'loss', 'tol': 0.08906732523947954, 'validation_fraction': 0.2, 'verbose': 4, 'warm_start': False}

Binning 0.100 GB of training data: 0.887 s
Binning 0.025 GB of validation data: 0.092 s
Fitting gradient boosted rounds:
[1/100000] 1 tree, 1352 leaves, max depth = 12, train loss: 4.40778, val loss: 4.41311, in 0.826s
[2/100000] 1 tree, 1371 leaves, max depth = 12, train loss: 4.40401, val loss: 4.40939, in 0.812s
[3/100000] 1 tree, 1364 leaves, max depth = 12, train loss: 4.40024, val loss: 4.40567, in 0.788s
[4/100000] 1 tree, 1368 leaves, max depth = 12, train loss: 4.39648, val loss: 4.40196, in 0.785s
[5/100000] 1 tree, 

[I 2021-11-27 13:54:14,732] Trial 1 finished with value: -3.9744934105310463 and parameters: {'learning_rate': 0.001415445349883366, 'max_depth': 12, 'l2_regularization': 0.02168029753677356, 'min_samples_leaf': 160, 'tol': 0.08906732523947954, 'n_iter_no_change': 79, 'random_state': 812124917}. Best is trial 0 with value: -3.8876998387783153.


...



Current model:

{'categorical_features': None, 'early_stopping': True, 'l2_regularization': 0.0020307363384580524, 'learning_rate': 0.002177150573098593, 'loss': 'absolute_error', 'max_bins': 255, 'max_depth': 6, 'max_iter': 100000, 'max_leaf_nodes': None, 'min_samples_leaf': 969, 'monotonic_cst': None, 'n_iter_no_change': 68, 'random_state': 192636529, 'scoring': 'loss', 'tol': 0.034981763833664935, 'validation_fraction': 0.2, 'verbose': 4, 'warm_start': False}

Binning 0.100 GB of training data: 0.833 s
Binning 0.025 GB of validation data: 0.084 s
Fitting gradient boosted rounds:
[1/100000] 1 tree, 60 leaves, max depth = 6, train loss: 4.40800, val loss: 4.41074, in 0.173s
[2/100000] 1 tree, 60 leaves, max depth = 6, train loss: 4.40394, val loss: 4.40669, in 0.149s
[3/100000] 1 tree, 60 leaves, max depth = 6, train loss: 4.39989, val loss: 4.40265, in 0.127s
[4/100000] 1 tree, 60 leaves, max depth = 6, train loss: 4.39585, val loss: 4.39862, in 0.151s
[5/100000] 1 tree, 60 leaves,

[I 2021-11-27 14:40:29,670] Trial 18 finished with value: -3.8515661205516736 and parameters: {'learning_rate': 0.002177150573098593, 'max_depth': 6, 'l2_regularization': 0.0020307363384580524, 'min_samples_leaf': 969, 'tol': 0.034981763833664935, 'n_iter_no_change': 68, 'random_state': 192636529}. Best is trial 16 with value: -3.8325659976867135.




Current model:

{'categorical_features': None, 'early_stopping': True, 'l2_regularization': 0.018764072014136098, 'learning_rate': 0.02418763183472459, 'loss': 'absolute_error', 'max_bins': 255, 'max_depth': 6, 'max_iter': 100000, 'max_leaf_nodes': None, 'min_samples_leaf': 764, 'monotonic_cst': None, 'n_iter_no_change': 47, 'random_state': 421717916, 'scoring': 'loss', 'tol': 0.03514932481541225, 'validation_fraction': 0.2, 'verbose': 4, 'warm_start': False}

Binning 0.100 GB of training data: 0.846 s
Binning 0.025 GB of validation data: 0.080 s
Fitting gradient boosted rounds:
[1/100000] 1 tree, 61 leaves, max depth = 6, train loss: 4.36613, val loss: 4.37465, in 0.362s
[2/100000] 1 tree, 62 leaves, max depth = 6, train loss: 4.32281, val loss: 4.33125, in 0.148s
[3/100000] 1 tree, 63 leaves, max depth = 6, train loss: 4.28046, val loss: 4.28884, in 0.152s
[4/100000] 1 tree, 64 leaves, max depth = 6, train loss: 4.23983, val loss: 4.24816, in 0.151s
[5/100000] 1 tree, 62 leaves, ma

[I 2021-11-27 14:41:07,768] Trial 19 finished with value: -3.874665679911936 and parameters: {'learning_rate': 0.02418763183472459, 'max_depth': 6, 'l2_regularization': 0.018764072014136098, 'min_samples_leaf': 764, 'tol': 0.03514932481541225, 'n_iter_no_change': 47, 'random_state': 421717916}. Best is trial 16 with value: -3.8325659976867135.


Best score: -3.8325659976867135

Optimized parameters: {'learning_rate': 0.0028710861658722024, 'max_depth': 5, 'l2_regularization': 6.541543606516337e-05, 'min_samples_leaf': 978, 'tol': 0.03877036102680547, 'n_iter_no_change': 71, 'random_state': 39084583}



### Model fitting

In [73]:
test_size = 0.15
val_size = 0.15
sample_size = 1.00
n_topics = 10
random_state = 0

model_inputs = get_model_inputs(val_size, test_size, sample_size, n_topics, random_state)

/home/alejandroxag/miniconda3/envs/ecommerce_jokes/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [74]:
fixed_params = {
                    'max_iter': 855,
                    'loss': 'absolute_error',
                    'max_leaf_nodes': None,
                    'early_stopping': False,
                    'verbose': 4,
                  }

print({**fixed_params,**hgb_params})

# Train
hgb = HistGradientBoostingRegressor(**{**fixed_params, **hgb_params})
hgb.fit(model_inputs['X_train'], model_inputs['y_train'])

# Test
y_true = model_inputs['y_test']
y_pred = hgb.predict(model_inputs['X_test'])
print(f'MAE: {mean_absolute_error(y_true, y_pred)}')

{'max_iter': 855, 'loss': 'absolute_error', 'max_leaf_nodes': None, 'early_stopping': False, 'verbose': 4, 'learning_rate': 0.0028710861658722024, 'max_depth': 5, 'l2_regularization': 6.541543606516337e-05, 'min_samples_leaf': 978, 'tol': 0.03877036102680547, 'n_iter_no_change': 71, 'random_state': 39084583}
Binning 0.591 GB of training data: 2.413 s
Fitting gradient boosted rounds:
[1/855] 1 tree, 32 leaves, max depth = 5, in 1.940s
[2/855] 1 tree, 32 leaves, max depth = 5, in 1.205s
[3/855] 1 tree, 32 leaves, max depth = 5, in 2.332s
[4/855] 1 tree, 32 leaves, max depth = 5, in 1.205s
[5/855] 1 tree, 32 leaves, max depth = 5, in 0.969s
[6/855] 1 tree, 32 leaves, max depth = 5, in 0.508s
[7/855] 1 tree, 32 leaves, max depth = 5, in 0.532s
[8/855] 1 tree, 32 leaves, max depth = 5, in 0.532s
[9/855] 1 tree, 32 leaves, max depth = 5, in 0.532s
[10/855] 1 tree, 32 leaves, max depth = 5, in 0.527s
[11/855] 1 tree, 32 leaves, max depth = 5, in 0.498s
[12/855] 1 tree, 32 leaves, max depth = 

# Recommendations to the users

### Unseen jokes per user

In [81]:
model_inputs['user_joke_info'][['user_id', 'joke_id'] + model_inputs['features']].columns

KeyboardInterrupt: 

In [ ]:
predictions_df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
predictions_df.insert(2, 'error', np.abs(predictions_df.y_true - predictions_df.y_pred))
display(predictions_df)